In [1]:
import datetime
print(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

2017-03-22 20:11:53


In [2]:
%gui qt
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os
import time
import tables as tb
import numpy as np

In [4]:
import invisible_cities.reco.tbl_functions as tbl
import invisible_cities.reco.wfm_functions as wfm
import invisible_cities.sierpe.blr as blr
import invisible_cities.reco.peak_functions_c as cpf
from invisible_cities.core.mpl_functions import set_plot_labels
from invisible_cities.reco.params import S12Params, ThresholdParams
from   invisible_cities.core.system_of_units_c import units

In [5]:
from plot_functions import plot_pmt_waveforms, plot_pmt_signals_vs_time_mus, plot_signal_vs_time_mus,\
                           plot_s12, hist_1d

In [6]:
from event_pmaps import EventPmaps, print_s12, print_s2si

In [7]:
from PyQt5 import QtCore, QtWidgets, uic
from matplotlib.backends.backend_qt5agg import (
    FigureCanvasQTAgg as FigureCanvas,
    NavigationToolbar2QT as NavigationToolbar)
from matplotlib.figure import Figure

from PyQt5.uic import loadUiType


qtCreatorFile = "event_maps.ui" # Enter file here.
Ui_MainWindow, QMainWindow = loadUiType(qtCreatorFile)

In [8]:
class QtGui(QMainWindow, Ui_MainWindow):
    def __init__(self, ):
        super(QtGui, self).__init__()
        self.setupUi(self)
        
        self.fig_dict = {}
        self.mplfigs.itemClicked.connect(self.change_figure)

        fig = Figure()
        self.add_mpl(fig)

    def add_figure(self, name, fig):
        test_key = self.fig_dict.pop(name, None)
        self.fig_dict[name] = fig
        if not test_key: # key not in dict
            self.mplfigs.addItem(name)

    def change_figure(self, item):
        text = item.text()
        self.rm_mpl()
        self.add_mpl(self.fig_dict[text])

    def add_mpl(self, fig):
        self.canvas = FigureCanvas(fig)
        self.mplvl.addWidget(self.canvas)
        self.canvas.draw()
        self.toolbar = NavigationToolbar(self.canvas,
                self.mplwindow, coordinates=True)
        self.mplvl.addWidget(self.toolbar)

    def rm_mpl(self,):
        self.mplvl.removeWidget(self.canvas)
        self.canvas.close()
        self.mplvl.removeWidget(self.toolbar)
        self.toolbar.close()



In [17]:
def scan_events(epm, gui, event_list=list(range(1)), event_plot_list=list(range(1)), printout=10):
    """Loop over events"""
    for event in event_list:
        if(event%printout == 0):
            print('processing event number {}'.format(event))
        epm.calibrated_sum(event)
        epm.find_s1(event)
        epm.find_s2(event)
        if (len(epm.S2) > 0):
            epm.find_s2si(event)
        
        if event in event_plot_list:
            #gui.add_figure('RWF-event-{}'.format(event), 
            #                plot_pmt_waveforms(epm.RWF, zoom=False, window_size=10000))
            gui.add_figure('CWF_vs_time_mus-event-{}'.format(event), 
                            plot_pmt_signals_vs_time_mus(epm.CWF,
                                                         epm.P.pmt_active,
                                                         t_min      =    0,
                                                         t_max      = 1300,
                                                         signal_min =    -5,
                                                         signal_max =  200))
            gui.add_figure('Calibrated_SUM-event-{}'.format(event), 
                            plot_signal_vs_time_mus(epm.csum, t_min=0, t_max=1300, signal_min=-5, signal_max=100))
            if (len(epm.S1) > 0):
                gui.add_figure('S1-event-{}'.format(event), plot_s12(epm.S1))
            if (len(epm.S2) > 0):
                gui.add_figure('S2-event-{}'.format(event), plot_s12(epm.S2))
                

In [22]:
RWF_file = os.path.join(os.environ['IC_DATA'],'LSC/3379/dst_waves.gdcsnext.000_3379.root.h5')


In [23]:
h5rwf = tb.open_file(RWF_file,'r')

In [24]:
pmtrwf, pmtblr, sipmrwf = tbl.get_vectors(h5rwf)

In [25]:
NEVT, NPMT, PMTWL = pmtrwf.shape
NEVT, NSIPM, SIPMWL = sipmrwf.shape
print("""
        Number of events in file = {}
        Number of PMTs = {}
        PMTWL = {}
        Number of SiPMs = {}
        SiPMWL = {}
      """.format(NEVT, NPMT, PMTWL,NSIPM, SIPMWL))


        Number of events in file = 163
        Number of PMTs = 12
        PMTWL = 52000
        Number of SiPMs = 1792
        SiPMWL = 1300
      


In [26]:
s1par  = S12Params(tmin=0*units.mus, tmax=640*units.mus, lmin=6, lmax=20, stride=4, rebin=False)
s2par = S12Params(tmin=640*units.mus, tmax=800*units.mus, stride=40, lmin=80, lmax=20000, rebin=True)
thr    = ThresholdParams(thr_s1=0.5 * units.pes,  thr_s2=1 *units.pes,
                        thr_MAU = 3 * units.adc, thr_sipm = 3.5 * units.pes,
                        thr_SIPM = 20 * units.adc)

In [27]:
gui=QtGui()
gui.show()
epm = EventPmaps(pmtrwf, sipmrwf, s1par, s2par, thr, verbose=False)

In [28]:
n_events = 160
sample_plot = 10
t0 = time.time()
scan_events(epm, gui, event_list=list(range(n_events)), event_plot_list=list(range(0,n_events,sample_plot)))
t1 = time.time()        
dt = t1 - t0
print("run {} events in {} s event/s = {}".format(n_events, dt, dt/n_events))

processing event number 0
processing event number 10
processing event number 20
processing event number 30
processing event number 40
processing event number 50
processing event number 60
processing event number 70
processing event number 80
processing event number 90
processing event number 100
processing event number 110
processing event number 120
processing event number 130
processing event number 140
processing event number 150
run 160 events in 65.28859543800354 s event/s = 0.4080537214875221


In [29]:
gui.add_figure('s1 peaks ',hist_1d(epm.s1f.peak(), xlo=0, xhi=max(epm.s1f.peak()) + 1))
gui.add_figure('s1 width in ns',hist_1d(epm.s1f.width(), xlo=0, xhi=500))
gui.add_figure('s1 t in mus',hist_1d(epm.s1f.tpeak()/units.mus, xlo=0, xhi=650))
gui.add_figure('s1 etot in pes',hist_1d(epm.s1f.etot(), xlo=0, xhi=max(epm.s1f.etot())+10))
gui.add_figure('s1 etot in pes (2)',hist_1d(epm.s1f.etot(), xlo=0, xhi=50))
gui.add_figure('s1 emax in pes',hist_1d(epm.s1f.emax(), xlo=0, xhi=max(epm.s1f.emax())+10))
gui.add_figure('s1 emax/etot in pes',hist_1d(epm.s1f.emax_over_etot(), xlo=0, xhi=1))

In [30]:
gui.add_figure('s2 peaks ',hist_1d(epm.s2f.peak(), xlo=0, xhi=max(epm.s2f.peak()) + 1))
gui.add_figure('s2 width in mua',hist_1d(epm.s2f.width()/units.mus, xlo=0, xhi=max(epm.s2f.width()/units.mus)+10))
gui.add_figure('s2 t in mus',hist_1d(epm.s2f.tpeak()/units.mus, xlo=650, xhi=670))
gui.add_figure('s2 etot in pes',hist_1d(epm.s2f.etot(), xlo=0, xhi=max(epm.s2f.etot())+100))
gui.add_figure('s2 emax in pes',hist_1d(epm.s2f.emax(), xlo=0, xhi=max(epm.s2f.emax())+100))
gui.add_figure('s2 emax/etot in pes',hist_1d(epm.s2f.emax_over_etot(), xlo=0, xhi=1))